In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23665193
paper_name = 'hirasawa_shimizu_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [95]:
original_data = pd.read_excel('raw_data/data_from_S1_PDF.xlsx', sheet_name='Mutants')

In [96]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 2415 x 12


In [97]:
original_data.head()

,eleted gene,Experiment 1 (mM),Experiment 2 (mM),Deleted gene,Experiment 1 (mM) .1,Experiment 2 (mM) .1,Deleted gene .1,Experiment 1 (mM) .2,Experiment 2 (mM) .2,Deleted gene .2,Experiment 1 (mM) .3,Experiment 2 (mM) .3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HFM1,1.6,258.8,APM3,144.1,163,YOL111C,118.7,128.8,MSS18,124,128.3
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZIP2,1.6,243.1,SKP2,147.1,128.2,REV1,149.5,139.4,MCH4,127.3,131.6
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
t1 = pd.melt(original_data.iloc[:,[0,3,6,9]])
t2 = pd.melt(original_data.iloc[:,[1,4,7,10]])
t3 = pd.melt(original_data.iloc[:,[2,5,8,11]])

In [99]:
original_data = pd.concat([t1,t2,t3], axis=1)

In [100]:
original_data = original_data.loc[original_data.iloc[:,1].notnull(),:]

In [101]:
original_data = original_data.iloc[:,[1,3,5]]
original_data.columns = ['genes','data1','data2']

In [102]:
original_data['genes'] = original_data['genes'].astype(str)

In [103]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [104]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['genes'].values, to='orf')

In [105]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            genes               data1               data2          orf
251   DELETEDGENE  Experiment 1 (mM)   Experiment 2 (mM)   DELETEDGENE
505   DELETEDGENE  Experiment 1 (mM)   Experiment 2 (mM)   DELETEDGENE
1012  DELETEDGENE  Experiment 1 (mM)   Experiment 2 (mM)   DELETEDGENE
2666  DELETEDGENE  Experiment 1 (mM)   Experiment 2 (mM)   DELETEDGENE
2920  DELETEDGENE  Experiment 1 (mM)   Experiment 2 (mM)   DELETEDGENE
3427  DELETEDGENE  Experiment 1 (mM)   Experiment 2 (mM)   DELETEDGENE
4123        SDC25               124.8               112.7        SDC25
5081  DELETEDGENE  Experiment 1 (mM)   Experiment 2 (mM)   DELETEDGENE
5289         CRS5               132.4               141.2         CRS5
5335  DELETEDGENE  Experiment 1 (mM)   Experiment 2 (mM)   DELETEDGENE
5437         AAD6               143.6               152.1         AAD6
5842  DELETEDGENE  Experiment 1 (mM)   Experiment 2 (mM)   DELETEDGENE
7308         FLO8               100.7               104.9         FLO8
7496  

In [106]:
original_data = original_data.loc[t,:]

In [107]:
original_data.set_index('orf', inplace=True)

In [108]:
original_data['data1'] = original_data['data1'].astype(float)
original_data['data2'] = original_data['data2'].astype(float)

In [111]:
original_data['data'] = original_data[['data1','data2']].mean(axis=1)

In [113]:
original_data = original_data[['data']].copy()

In [114]:
original_data = original_data.groupby(original_data.index).mean()

In [115]:
original_data.shape

(4717, 1)

# Normalize by controls

In [116]:
ctrl_data = pd.read_excel('raw_data/data_from_S1_PDF.xlsx', sheet_name='CTRL')

In [117]:
print('Data dimensions: %d x %d' % (ctrl_data.shape))

Data dimensions: 133 x 12


In [118]:
t2 = pd.melt(ctrl_data.iloc[:,[1,2,4,5,7,8,10,11]])

In [119]:
ctrl_data = t2['value'].mean()

In [120]:
original_data = original_data / ctrl_data

# Prepare the final dataset

In [122]:
data = original_data.copy()

In [123]:
dataset_ids = [94]
datasets = datasets.reindex(index=dataset_ids)

In [124]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [125]:
data.head()

dataset_id,94
data_type,value
orf,
YAL002W,0.915417
YAL004W,1.064283
YAL005C,1.147974
YAL007C,0.146274
YAL008W,1.108721


## Subset to the genes currently in SGD

In [126]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 20


In [127]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,94
,data_type,value
gene_id,orf,
2,YAL002W,0.915417
1863,YAL004W,1.064283
4,YAL005C,1.147974
5,YAL007C,0.146274
6,YAL008W,1.108721


# Normalize

In [128]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [129]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [130]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id             94          
data_type           value    valuez
gene_id orf                        
2       YAL002W  0.915417 -0.441085
1863    YAL004W  1.064283  0.289857
4       YAL005C  1.147974  0.700785
5       YAL007C  0.146274 -4.217622
6       YAL008W  1.108721  0.508049

# Print out

In [131]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [132]:
from IO.save_data_to_db3 import *

In [133]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 23665193...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.89s/it]

Updating the data_modified_on field...
